In [164]:
import distributed
c = distributed.Client('172.31.7.105:8786')
c

<Client: scheduler="172.31.7.105:8786" processes=2 cores=2>

In [133]:
#!pip install siphon

In [173]:
from subprocess import call

def install_siphon():
    return call(["pip", "install", "siphon"])

c.run(install_siphon)

{'172.31.30.77:22472': 0, '172.31.30.77:24560': 0}

In [186]:
from siphon.catalog import TDSCatalog

In [187]:
def get_catalog(url):
    return TDSCatalog(url)

url = 'http://localhost/thredds/catalog.xml'
catalog = c.submit(get_catalog, url).result()
catalog

In [188]:
catalog.catalog_refs

OrderedDict([('S3 Mogreps G', <siphon.catalog.CatalogRef at 0x7f541ffd9358>),
             ('S3 Mogreps UK', <siphon.catalog.CatalogRef at 0x7f541ffd9fd0>)])

In [189]:
mogreps_g = catalog.catalog_refs['S3 Mogreps G']
mogreps_uk = catalog.catalog_refs['S3 Mogreps UK']

In [190]:
g_catalog = c.submit(get_catalog, mogreps_g.href).result()
uk_catalog = c.submit(get_catalog, mogreps_uk.href).result()
g_catalog

In [206]:
uk_catalog.datasets

OrderedDict([('prods_op_mogreps-uk_20130104_21_08_006.nc',
              <siphon.catalog.Dataset at 0x7f54401cd2b0>),
             ('prods_op_mogreps-uk_20130104_21_07_036.nc',
              <siphon.catalog.Dataset at 0x7f54400f6d30>),
             ('prods_op_mogreps-uk_20130104_21_07_033.nc',
              <siphon.catalog.Dataset at 0x7f54400f6c88>),
             ('prods_op_mogreps-uk_20130104_21_07_030.nc',
              <siphon.catalog.Dataset at 0x7f54400f6550>),
             ('prods_op_mogreps-uk_20130104_21_07_027.nc',
              <siphon.catalog.Dataset at 0x7f54400f6208>),
             ('prods_op_mogreps-uk_20130104_21_07_021.nc',
              <siphon.catalog.Dataset at 0x7f54400f6470>),
             ('prods_op_mogreps-uk_20130104_21_07_018.nc',
              <siphon.catalog.Dataset at 0x7f54400f6a90>),
             ('prods_op_mogreps-uk_20130104_21_06_036.nc',
              <siphon.catalog.Dataset at 0x7f54400f6940>),
             ('prods_op_mogreps-uk_20130104_21_06_030.nc

In [205]:
uk_sample = uk_catalog.datasets['prods_op_mogreps-uk_20160101_03_00_003.nc']
uk_sample

KeyError: 'prods_op_mogreps-uk_20160101_03_00_003.nc'

In [194]:
uk_sample.access_urls

{'OPENDAP': 'http://localhost/thredds/dodsC/mogreps/uk/S3/prods_op_mogreps-uk_20130104_21_08_006.nc'}

In [210]:
g_sample = g_catalog.datasets['prods_op_mogreps-g_20160101_00_00_003.nc']
g_sample

In [157]:
dataset = catalog3.datasets['20160101_06_mogreps-g_dew_point_temperature.nc']
dataset.access_urls

{'OPENDAP': 'http://localhost/thredds/dodsC/mogreps/g/S3/dew_point_temperature/20160101_06_mogreps-g_dew_point_temperature.nc'}

In [158]:
import iris

cube = c.submit(iris.load_cube, dataset.access_urls['OPENDAP']).result()
cube

<iris 'Cube' of dew_point_temperature / (K) (realization: 12; time: 58; latitude: 600; longitude: 800)>

In [159]:
print(cube)

dew_point_temperature / (K)         (realization: 12; time: 58; latitude: 600; longitude: 800)
     Dimension coordinates:
          realization                           x         -             -               -
          time                                  -         x             -               -
          latitude                              -         -             x               -
          longitude                             -         -             -               x
     Auxiliary coordinates:
          forecast_period                       -         x             -               -
     Scalar coordinates:
          forecast_reference_time: 2016-01-01 06:00:00
          height: 1.5 m
     Attributes:
          Conventions: CF-1.5
          DODS_EXTRA.Unlimited_Dimension: realization
          STASH: m01s03i250
          _ChunkSizes: [  1  15 200 267]
          source: Data from Met Office Unified Model
          um_version: 10.1


In [160]:
urls = list(map(lambda x: list(x)[1].access_urls['OPENDAP'], catalog3.datasets.items()))
urls[:4]

['http://localhost/thredds/dodsC/mogreps/g/S3/dew_point_temperature/20160105_18_mogreps-g_dew_point_temperature.nc',
 'http://localhost/thredds/dodsC/mogreps/g/S3/dew_point_temperature/20160105_12_mogreps-g_dew_point_temperature.nc',
 'http://localhost/thredds/dodsC/mogreps/g/S3/dew_point_temperature/20160105_06_mogreps-g_dew_point_temperature.nc',
 'http://localhost/thredds/dodsC/mogreps/g/S3/dew_point_temperature/20160105_00_mogreps-g_dew_point_temperature.nc']

In [161]:
cubes = c.submit(iris.load, urls).result()
cubes

[<iris 'Cube' of dew_point_temperature / (K) (realization: 12; time: 58; latitude: 600; longitude: 800)>,
<iris 'Cube' of dew_point_temperature / (K) (realization: 12; time: 58; latitude: 600; longitude: 800)>,
<iris 'Cube' of dew_point_temperature / (K) (realization: 12; time: 58; latitude: 600; longitude: 800)>,
<iris 'Cube' of dew_point_temperature / (K) (realization: 12; time: 58; latitude: 600; longitude: 800)>,
<iris 'Cube' of dew_point_temperature / (K) (realization: 12; time: 58; latitude: 600; longitude: 800)>,
<iris 'Cube' of dew_point_temperature / (K) (realization: 12; time: 58; latitude: 600; longitude: 800)>,
<iris 'Cube' of dew_point_temperature / (K) (realization: 12; time: 58; latitude: 600; longitude: 800)>,
<iris 'Cube' of dew_point_temperature / (K) (realization: 12; time: 58; latitude: 600; longitude: 800)>,
<iris 'Cube' of dew_point_temperature / (K) (realization: 12; time: 58; latitude: 600; longitude: 800)>,
<iris 'Cube' of dew_point_temperature / (K) (realizati

In [102]:
cubes

[<iris 'Cube' of dew_point_temperature / (K) (realization: 12; time: 58; latitude: 600; longitude: 800)>,
<iris 'Cube' of dew_point_temperature / (K) (realization: 12; time: 58; latitude: 600; longitude: 800)>,
<iris 'Cube' of dew_point_temperature / (K) (realization: 12; time: 58; latitude: 600; longitude: 800)>,
<iris 'Cube' of dew_point_temperature / (K) (realization: 12; time: 58; latitude: 600; longitude: 800)>,
<iris 'Cube' of dew_point_temperature / (K) (realization: 12; time: 58; latitude: 600; longitude: 800)>,
<iris 'Cube' of dew_point_temperature / (K) (realization: 12; time: 58; latitude: 600; longitude: 800)>,
<iris 'Cube' of dew_point_temperature / (K) (realization: 12; time: 58; latitude: 600; longitude: 800)>,
<iris 'Cube' of dew_point_temperature / (K) (realization: 12; time: 58; latitude: 600; longitude: 800)>,
<iris 'Cube' of dew_point_temperature / (K) (realization: 12; time: 58; latitude: 600; longitude: 800)>,
<iris 'Cube' of dew_point_temperature / (K) (realizati

In [93]:
wbpt = c.submit(get_catalog, 'http://localhost/thredds/catalog/mogreps/g/S3/wet_bulb_potential_temperature/catalog.xml').result()
wbpt

In [100]:
urls2 = list(map(lambda x: list(x)[1].access_urls['OPENDAP'], wbpt.datasets.items()))
urls2[:4]

['http://localhost/thredds/dodsC/mogreps/g/S3/wet_bulb_potential_temperature/20160105_18_mogreps-g_wet_bulb_potential_temperature.nc',
 'http://localhost/thredds/dodsC/mogreps/g/S3/wet_bulb_potential_temperature/20160105_12_mogreps-g_wet_bulb_potential_temperature.nc',
 'http://localhost/thredds/dodsC/mogreps/g/S3/wet_bulb_potential_temperature/20160105_06_mogreps-g_wet_bulb_potential_temperature.nc',
 'http://localhost/thredds/dodsC/mogreps/g/S3/wet_bulb_potential_temperature/20160105_00_mogreps-g_wet_bulb_potential_temperature.nc']

In [98]:
cubes2 = c.submit(iris.load, urls2).result()

In [99]:
cubes2

[<iris 'Cube' of wet_bulb_potential_temperature / (K) (realization: 12; time: 59; pressure: 3; latitude: 600; longitude: 800)>,
<iris 'Cube' of wet_bulb_potential_temperature / (K) (realization: 12; time: 59; pressure: 3; latitude: 600; longitude: 800)>,
<iris 'Cube' of wet_bulb_potential_temperature / (K) (realization: 12; time: 59; pressure: 3; latitude: 600; longitude: 800)>,
<iris 'Cube' of wet_bulb_potential_temperature / (K) (realization: 12; time: 59; pressure: 3; latitude: 600; longitude: 800)>,
<iris 'Cube' of wet_bulb_potential_temperature / (K) (realization: 12; time: 59; pressure: 3; latitude: 600; longitude: 800)>,
<iris 'Cube' of wet_bulb_potential_temperature / (K) (realization: 12; time: 59; pressure: 3; latitude: 600; longitude: 800)>,
<iris 'Cube' of wet_bulb_potential_temperature / (K) (realization: 12; time: 59; pressure: 3; latitude: 600; longitude: 800)>,
<iris 'Cube' of wet_bulb_potential_temperature / (K) (realization: 12; time: 59; pressure: 3; latitude: 600; l

In [ ]:
for i in catalog2.catalog_refs.items():
    cat_ref = i[1]
    new_cat = c.submit(get_catalog, cat_ref.href).result()
    urls = list(map(lambda x: list(x)[1].access_urls['OPENDAP'], new_cat.datasets.items()))
    print(c.submit(iris.load, urls).result())

RuntimeError: NetCDF: file not found